In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import time
from bs4 import BeautifulSoup

In [2]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

In [3]:
bestsellers = pd.read_csv(r"C:\Users\melan\Documents\DA8\Projects\bestseller_capstone\data\all_bestsellers.csv")

C:\Users\melan\AppData\Local\Temp\ipykernel_27672\3798271838.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  bestsellers = pd.read_csv(r"C:\Users\melan\Documents\DA8\Projects\bestseller_capstone\data\all_bestsellers.csv")


In [4]:
bestsellers.head()

,title,description,author,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,week,...,title_y,author_y,avg_rating,count_rating,count_reviews,date_published,top_genre,genre1,genre2,genre3
0,Full Package,A man shares a cramped apartment with his frie...,Lauren Blakely,Lauren Blakely,A00B01MT5HMRV,None,1,0,1,2017-01-25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,The Guernsey Literary And Potato Peel Pie Society,"After World War II, a journalist travels to th...",Mary Ann Shaffer and Annie Barrows,Dial,9780440337973,0440337976,2,0,2,2017-01-25,...,The Guernsey Literary and Potato Peel Pie Society,Mary Ann Shaffer,4.19,721465.0,60356.0,2008-07-29,"Historical Fiction, Fiction, Romance, Historic...",Historical Fiction,Fiction,Romance
2,The Whistler,A whistleblower alerts a Florida investigator ...,John Grisham,Doubleday,9780385541206,0385541201,3,4,12,2017-01-25,...,The Whistler,John Grisham,3.95,120472.0,7710.0,2016-10-25,"Fiction, Mystery, Thriller, Audiobook, Crime, ...",Fiction,Mystery,Thriller
3,Ring Of Fire,"Pike Logan, a member of a secret counterterror...",Brad Taylor,Dutton,9781101984772,None,4,0,1,2017-01-25,...,Ring of Fire,Brad Taylor,4.28,4371.0,230.0,2017-01-10,"Thriller, Fiction, Action, Military Fiction, S...",Thriller,Fiction,Action
4,Small Great Things,"A medical crisis entangles a black nurse, a wh...",Jodi Picoult,Ballantine,9780345544964,034554496X,5,7,9,2017-01-25,...,Small Great Things,Jodi Picoult,4.35,342790.0,30250.0,2016-10-11,"Fiction, Contemporary, Audiobook, Adult, Adult...",Fiction,Contemporary,Audiobook


In [5]:
#identify rows with missing goodreads data
goodreads_nulls = bestsellers[bestsellers['title_y'].isnull()]
len(goodreads_nulls)

6843

In [6]:
#set up chrome, sign in to goodreads
s = Service("chromedriver.exe")
opts= Options()
opts.headless = True
driver = webdriver.Chrome(service=s)
driver.get("https://www.goodreads.com")
#find sign in link
link = driver.find_element(By.LINK_TEXT, 'Sign In')
link.click()
#sign in with email
link2 = driver.find_element(By.LINK_TEXT, 'Sign in with email')
link2.click()
#enter username and pwd
log_email = driver.find_element(By.NAME, 'email')
log_pwd = driver.find_element(By.NAME, 'password')
log_email.send_keys("melaniedwesson@gmail.com")
log_pwd.send_keys("qK5YGNb4x4_Vp3G")
log_pwd.submit
#click sign in button
button1 = driver.find_element(By.ID, 'signInSubmit')
button1.click()

C:\Users\melan\AppData\Local\Temp\ipykernel_27672\1531218307.py:4: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  opts.headless = True


In [9]:
#try a search by title - tried doing a title/author combo earlier and was getting too many errors
#get unique titles, turn into a list
unique_titles = goodreads_nulls['title'].unique()
book_list = unique_titles.tolist()
book_list


['Full Package',
 'Shelter For Adeline',
 'Until Ashlyn',
 'Melt',
 'Craving',
 'A Game Of Thrones',
 'Over The Edge',
 'Managed',
 'Rescuing Harley',
 'Wrong',
 'The Sexy One',
 'The Fix Up',
 'Possession',
 'Playboy Pilot',
 'A Summer In Sonoma',
 'Well Hung',
 'Anti-Stepbrother',
 'The Memory Box',
 'A Beautiful Funeral',
 'Shadow Born',
 'Midnight Soul',
 'Every Beat Of My Heart',
 'Smooth Operator',
 'Magic And Mayhem',
 'Scorch',
 'Obsession',
 'Rescuing Emily',
 'Unexpected Reality',
 'Hitched',
 'Easy Kisses',
 'Until June',
 'Six John Jordan Mysteries',
 'Thirty-Six And A Half Motives',
 'Mister O',
 'The Winslow Brothers Boxed Set',
 'Stuck-Up Suit',
 'Logan Kade',
 'The Arrangement 21 (The Ferro Family)',
 'Roomhate',
 'The Lie',
 'January',
 'April',
 'The Baller',
 'February',
 'Sebring',
 'Big Rock',
 'March',
 'Seducing Simon',
 'Monster ________',
 'Easy Melody',
 'Thirty-Five And A Half Consipiracies',
 'Hard Beat',
 'Kian',
 'Wicked After Dark',
 'Until Jax',
 'Screwe

In [14]:
# Set up scraper loop
self_published = []
unavailable = []

# Loop through book_list
for title in book_list:
    try:
        # Look up each book in the search bar
        search_book = driver.find_element(By.NAME, "q")
        search_book.clear()  # Clear any previous search
        search_book.send_keys(title)  # Use 'title' variable instead of undefined 'item'
        search_book.submit()

        # Wait for search results to load
        time.sleep(3)

        #Click cover art
        cover_element = driver.find_element(By.CLASS_NAME, "bookCover")
        cover_element.click()

        #Wait for results to load
        time.sleep(3)

        # Scrape the desired information
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
        gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
        avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
        count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
        count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
        date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

        # Scrape the genre
        genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
        genres = [genre.text.strip() for genre in genre_elements]
        genre1 = ', '.join(genres) if genres else None

        # Append scraped data to the self_published list
        self_published.append({
            'title': gr_title,
            'author': gr_author,
            'avg_rating': avg_rating,
            'count_rating': count_rating,
            'count_reviews': count_reviews,
            'date_published': date_published,
            'top_genre': genre1
        })

    except Exception as e:
        # Handle the exception here
        unavailable.append({
            'title': title,  # Use 'title' variable instead of undefined 'gr_title'
            'author': None  # Set author as None if not found
        })


In [15]:
len(self_published)

10

In [16]:
len(unavailable)

0

In [17]:
#Got some weird results in there, maybe I should try having the scraper find the link with the matching title
# Set up scraper loop
self_published = []
unavailable = []

# Loop through book_list
for title in book_list[:10]:
    try:
        # Look up each book in the search bar
        search_book = driver.find_element(By.NAME, "q")
        search_book.clear()  # Clear any previous search
        search_book.send_keys(title)
        search_book.submit()

        # Wait for search results to load
        time.sleep(3)

        # Find the link with a matching title
        link_element = driver.find_element(By.XPATH, f"//a[contains(text(), '{title}')]")
        link_element.click()

        # Wait for the book page to load
        time.sleep(3)

        # Scrape the desired information
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
        gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
        avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
        count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
        count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
        date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

        # Scrape the genre
        genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
        genres = [genre.text.strip() for genre in genre_elements]
        genre1 = ', '.join(genres) if genres else None

        # Append scraped data to the self_published list
        self_published.append({
            'title': gr_title,
            'author': gr_author,
            'avg_rating': avg_rating,
            'count_rating': count_rating,
            'count_reviews': count_reviews,
            'date_published': date_published,
            'top_genre': genre1
        })

    except Exception as e:
        # Handle the exception here
        unavailable.append({
            'title': title,
            'author': None  # Set author as None if not found
        })


KeyboardInterrupt: 

In [18]:
#Try again with XPATH + span
# Set up scraper loop
self_published = []
unavailable = []

# Loop through book_list
for title in book_list[:10]:
    try:
        # Look up the book page URL for the given title
        search_book = driver.find_element(By.NAME, "q")
        search_book.clear()  # Clear any previous search
        search_book.send_keys(title)  # Use 'title' variable instead of undefined 'item'
        search_book.submit()

        # Wait for search results to load
        time.sleep(3)

        # Find the link with a matching title
        link_element = driver.find_element(By.XPATH, f"//span[@itemprop='name' and contains(text(), '{title}')]/ancestor::a")
        link_element.click()

        # Wait for the book page to load
        time.sleep(3)

        # Scrape the desired information
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
        gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
        avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
        count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
        count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
        date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

        # Scrape the genre
        genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
        genres = [genre.text.strip() for genre in genre_elements]
        genre1 = ', '.join(genres) if genres else None

        # Append scraped data to the self_published list
        self_published.append({
            'title': gr_title,
            'author': gr_author,
            'avg_rating': avg_rating,
            'count_rating': count_rating,
            'count_reviews': count_reviews,
            'date_published': date_published,
            'top_genre': genre1
        })

    except Exception as e:
        # Handle the exception here
        unavailable.append({
            'title': title,  # Use 'title' variable instead of undefined 'gr_title'
            'author': None  # Set author as None if not found
        })


In [20]:
#Still getting some weird results.  Need to search for both title and author before clicking link.  Make a new list of authors
#try a search by title - tried doing a title/author combo earlier and was getting too many errors
#get unique titles, turn into a list

title_author_pairs = list(zip(goodreads_nulls['title'], goodreads_nulls['author']))

In [24]:
# Get unique title_author pairs using set()
unique_title_author_pairs = set(title_author_pairs)

# Convert the unique pairs back to a list
unique_title_author_pairs_list = list(unique_title_author_pairs)

#count
len(unique_title_author_pairs)


1064

In [25]:
#try again using unique title/author pairs


KeyboardInterrupt: 

In [26]:
#try again with both title/author in link_element
# Set up scraper loop
self_published = []
unavailable = []

# Loop through unique_title_author_pairs
for title, author in unique_title_author_pairs:
    try:
        # Look up the book page URL for the given title
        search_book = driver.find_element(By.NAME, "q")
        search_book.clear()  # Clear any previous search
        search_book.send_keys(title)  # Use title as the search query
        search_book.submit()

        # Wait for search results to load
        time.sleep(3)

        # Find the link with matching title and author
        link_element = driver.find_element(By.XPATH, f"//a[span[@itemprop='name' and contains(text(), '{title}') and contains(text(), '{author}')]]")
        link_element.click()

        # Wait for the book page to load
        time.sleep(3)

        # Scrape the desired information
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
        gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
        avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
        count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
        count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
        date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

        # Scrape the genre
        genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
        genres = [genre.text.strip() for genre in genre_elements]
        genre1 = ', '.join(genres) if genres else None

        # Append scraped data to the self_published list
        self_published.append({
            'title': gr_title,
            'author': gr_author,
            'avg_rating': avg_rating,
            'count_rating': count_rating,
            'count_reviews': count_reviews,
            'date_published': date_published,
            'top_genre': genre1
        })

    except Exception as e:
        # Handle the exception here
        unavailable.append({
            'title': title,
            'author': author
        })


KeyboardInterrupt: 

In [29]:
#last one wasn't clicking the correct links. try again looping through search results to compare title/author
# Set up scraper loop
self_published = []
unavailable = []

# Loop through unique_title_author_pairs
for title, author in unique_title_author_pairs:
    try:
        # Look up the book page URL for the given title
        search_book = driver.find_element(By.NAME, "q")
        search_book.clear()  # Clear any previous search
        search_book.send_keys(title)  # Use title as the search query
        search_book.submit()

        # Wait for search results to load
        time.sleep(3)

        # Find all search results
        search_results = driver.find_elements(By.XPATH, "//tr[contains(@id, 'bookTitle')]")

        # Loop through search results to find the matching link
        for result in search_results:
            result_title = result.find_element(By.XPATH, ".//a/span[@itemprop='name']").text.strip()
            result_author = result.find_element(By.XPATH, ".//span[@itemprop='name']").text.strip()

            # Compare the title and author of the result with the desired values
            if result_title == title and result_author == author:
                link_element = result.find_element(By.XPATH, ".//a")
                link_element.click()
                break  # Exit the loop if a match is found

        # Wait for the book page to load
        time.sleep(3)

        # Scrape the desired information
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
        gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
        avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
        count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
        count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
        date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

        # Scrape the genre
        genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
        genres = [genre.text.strip() for genre in genre_elements]
        genre1 = ', '.join(genres) if genres else None

        # Append scraped data to the self_published list
        self_published.append({
            'title': gr_title,
            'author': gr_author,
            'avg_rating': avg_rating,
            'count_rating': count_rating,
            'count_reviews': count_reviews,
            'date_published': date_published,
            'top_genre': genre1
        })

    except Exception as e:
        # Handle the exception here
        unavailable.append({
            'title': title,
            'author': author
        })


KeyboardInterrupt: 

In [30]:
#last one still wasn't clicking anything, just searching all the names
#try again using find_element_by_link_text

# Set up scraper loop
self_published = []
unavailable = []

# Loop through unique_title_author_pairs
for title, author in unique_title_author_pairs:
    try:
        # Look up the book page URL for the given title
        search_book = driver.find_element(By.NAME, "q")
        search_book.clear()  # Clear any previous search
        search_book.send_keys(title)  # Use title as the search query
        search_book.submit()

        # Wait for search results to load
        time.sleep(3)

        # Find the link with matching title and author
        link_text = f"{title} ({author})"
        link_element = driver.find_element_by_link_text(link_text)
        link_element.click()

        # Wait for the book page to load
        time.sleep(3)

        # Scrape the desired information
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
        gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
        avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
        count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
        count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
        date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

        # Scrape the genre
        genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
        genres = [genre.text.strip() for genre in genre_elements]
        genre1 = ', '.join(genres) if genres else None

        # Append scraped data to the self_published list
        self_published.append({
            'title': gr_title,
            'author': gr_author,
            'avg_rating': avg_rating,
            'count_rating': count_rating,
            'count_reviews': count_reviews,
            'date_published': date_published,
            'top_genre': genre1
        })

    except Exception as e:
        # Handle the exception here
        unavailable.append({
            'title': title,
            'author': author
        })


KeyboardInterrupt: 

In [31]:
#try again clicking on cover art
# Set up scraper loop
self_published = []
unavailable = []

# Loop through unique_title_author_pairs
for title, author in unique_title_author_pairs:
    try:
        # Look up the book page URL for the given title
        search_book = driver.find_element(By.NAME, "q")
        search_book.clear()  # Clear any previous search
        search_book.send_keys(title)  # Use title as the search query
        search_book.submit()

        # Wait for search results to load
        time.sleep(3)

        # Find the cover art element with a partial match for the alt attribute value
        cover_element = driver.find_element(By.XPATH, f"//img[contains(@alt, '{title}') and contains(@alt, '{author}')]")

        # Click on the cover art element
        cover_element.click()

        # Wait for the book page to load
        time.sleep(3)

        # Scrape the desired information
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
        gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
        avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
        count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
        count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
        date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

        # Scrape the genre
        genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
        genres = [genre.text.strip() for genre in genre_elements]
        genre1 = ', '.join(genres) if genres else None

        # Append scraped data to the self_published list
        self_published.append({
            'title': gr_title,
            'author': gr_author,
            'avg_rating': avg_rating,
            'count_rating': count_rating,
            'count_reviews': count_reviews,
            'date_published': date_published,
            'top_genre': genre1
        })

    except Exception as e:
        # Handle the exception here
        unavailable.append({
            'title': title,
            'author': author
        })


KeyboardInterrupt: 

In [32]:
#same problem, try find_element_by_partial_link_text method
# Set up scraper loop
self_published = []
unavailable = []

# Loop through unique_title_author_pairs
for title, author in unique_title_author_pairs:
    try:
        # Look up the book page URL for the given title
        search_book = driver.find_element(By.NAME, "q")
        search_book.clear()  # Clear any previous search
        search_book.send_keys(title)  # Use title as the search query
        search_book.submit()

        # Wait for search results to load
        time.sleep(3)

        # Find the link element with partial text matching the title and author
        link_element = driver.find_element(By.PARTIAL_LINK_TEXT, f"{title} ({author}")

        # Click on the link element
        link_element.click()

        # Wait for the book page to load
        time.sleep(3)

        # Scrape the desired information
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
        gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
        avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
        count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
        count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
        date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

        # Scrape the genre
        genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
        genres = [genre.text.strip() for genre in genre_elements]
        genre1 = ', '.join(genres) if genres else None

        # Append scraped data to the self_published list
        self_published.append({
            'title': gr_title,
            'author': gr_author,
            'avg_rating': avg_rating,
            'count_rating': count_rating,
            'count_reviews': count_reviews,
            'date_published': date_published,
            'top_genre': genre1
        })

    except Exception as e:
        # Handle the exception here
        unavailable.append({
            'title': title,
            'author': author
        })


KeyboardInterrupt: 

In [33]:
#nope same problem.  Try finding all the link elements on the page and iterate through to find a match based on both title and author
# Set up scraper loop
self_published = []
unavailable = []

# Loop through unique_title_author_pairs
for title, author in unique_title_author_pairs:
    try:
        # Look up the book page URL for the given title
        search_book = driver.find_element(By.NAME, "q")
        search_book.clear()  # Clear any previous search
        search_book.send_keys(title)  # Use title as the search query
        search_book.submit()

        # Wait for search results to load
        time.sleep(3)

        # Find all the link elements on the search results page
        link_elements = driver.find_elements(By.TAG_NAME, "a")

        # Iterate through the link elements
        for link_element in link_elements:
            # Check if the link element text contains both the title and author
            if title in link_element.text and author in link_element.text:
                # Click on the link element
                link_element.click()
                break

        # Wait for the book page to load
        time.sleep(3)

        # Scrape the desired information
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
        gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
        avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
        count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
        count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
        date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

        # Scrape the genre
        genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
        genres = [genre.text.strip() for genre in genre_elements]
        genre1 = ', '.join(genres) if genres else None

        # Append scraped data to the self_published list
        self_published.append({
            'title': gr_title,
            'author': gr_author,
            'avg_rating': avg_rating,
            'count_rating': count_rating,
            'count_reviews': count_reviews,
            'date_published': date_published,
            'top_genre': genre1
        })

    except Exception as e:
        # Handle the exception here
        unavailable.append({
            'title': title,
            'author': author
        })


KeyboardInterrupt: 

In [37]:
#didn't work because it was looking for both title/author when only title is in that link elem3ent.  
#try again to search for title only in link_element text and then check if author is present on the book page

# Set up scraper loop
self_published = []
unavailable = []

# Loop through unique_title_author_pairs
for title, author in unique_title_author_pairs:
    try:
        # Look up the book page URL for the given title
        search_book = driver.find_element(By.NAME, "q")
        search_book.clear()  # Clear any previous search
        search_book.send_keys(title)  # Use title as the search query
        search_book.submit()

        # Wait for search results to load
        time.sleep(3)

        # Find all the link elements on the search results page
        link_elements = driver.find_elements(By.TAG_NAME, "a")

        # Iterate through the link elements
        for link_element in link_elements:
            # Check if the title is present in the link element text
            if title in link_element.text:
                # Click on the link element
                link_element.click()
                break

        # Wait for the book page to load
        time.sleep(3)

        # Check if the author is present on the book page
        author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author}')]")
        if author_element:
            # Scrape the desired information
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
            gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
            avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
            count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
            count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
            date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

            # Scrape the genre
            genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
            genres = [genre.text.strip() for genre in genre_elements]
            genre1 = ', '.join(genres) if genres else None

            # Append scraped data to the self_published list
            self_published.append({
                'title': gr_title,
                'author': gr_author,
                'avg_rating': avg_rating,
                'count_rating': count_rating,
                'count_reviews': count_reviews,
                'date_published': date_published,
                'top_genre': genre1
            })
        else:
            # Handle the case when the author is not found on the book page
            unavailable.append({
                'title': title,
                'author': author
            })

    except Exception as e:
        # Handle the exception here
        unavailable.append({
            'title': title,
            'author': author
        })


In [38]:
len(self_published)

71

In [39]:
len(unavailable)

993

In [41]:
unavailable

[{'title': 'Craving', 'author': 'Helen Hardt'},
 {'title': 'Wyoming Heart', 'author': 'Diana Palmer'},
 {'title': 'State Of Terror',
  'author': 'Hillary Rodham Clinton and Louise Penny'},
 {'title': 'Kiss Of Christmas Magic',
  'author': 'Eve Langlais, Aubrey Rose, Molly Prince and others'},
 {'title': 'Wrong', 'author': 'Jana Aston'},
 {'title': 'A Feast For Crows', 'author': 'George RR Martin'},
 {'title': 'Over The Edge', 'author': 'Meredith Wild'},
 {'title': 'Sweet Possession', 'author': 'J Daniels'},
 {'title': 'Wild Like The Wind', 'author': 'Kristen Ashley'},
 {'title': 'Down And Out', 'author': 'Kelley R Martin'},
 {'title': 'The Spanish Love Deception', 'author': 'Elena Armas'},
 {'title': 'Fated Mates, The Alpha Shifter Boxed Set',
  'author': 'Skye Eagleday'},
 {'title': 'Shadow Tyrants', 'author': 'Clive Cussler and Boyd Morrison'},
 {'title': 'The Persian Gamble', 'author': 'Joel C Rosenberg'},
 {'title': 'Ravenous', 'author': 'Helen Hardt'},
 {'title': 'Mastered', 'auth

In [42]:
self_published

[{'title': 'The Arrangement 10: The Ferro Family',
  'author': 'H.M. Ward',
  'avg_rating': '4.19',
  'count_rating': '7977',
  'count_reviews': '475',
  'date_published': 'First published September 22, 2013',
  'top_genre': 'Genres, Romance, Erotica, New Adult, Contemporary, Adult, Contemporary Romance, Drama'},
 {'title': 'One Minute Out',
  'author': 'Mark Greaney',
  'avg_rating': '4.45',
  'count_rating': '11071',
  'count_reviews': '630',
  'date_published': 'First published February 18, 2020',
  'top_genre': 'Genres, Thriller, Fiction, Action, Suspense, Audiobook, Espionage, Mystery Thriller'},
 {'title': 'Dark Tarot',
  'author': 'Christine Feehan',
  'avg_rating': '4.52',
  'count_rating': '4163',
  'count_reviews': '517',
  'date_published': 'First published November 9, 2021',
  'top_genre': 'Genres, Paranormal, Romance, Paranormal Romance, Fantasy, Vampires, Demons, Dark'},
 {'title': None,
  'author': None,
  'avg_rating': None,
  'count_rating': None,
  'count_reviews': No

In [45]:
#so a lot of these are still being flagged as unavailable, but I see them on goodreads.  Let's try using lower and in

self_published = []
unavailable = []

# Convert unique_title_author_pairs to a list
unique_pairs_list = list(unique_title_author_pairs)

# Loop through the pairs using a counter variable
counter = 0
for title, author in unique_pairs_list:
    if counter >= 10:
        break
    
    try:
        # Look up the book page URL for the given title
        search_book = driver.find_element(By.NAME, "q")
        search_book.clear()  # Clear any previous search
        search_book.send_keys(title)  # Use title as the search query
        search_book.submit()

        # Wait for search results to load
        time.sleep(3)

        # Find all the link elements on the search results page
        link_elements = driver.find_elements(By.TAG_NAME, "a")

        # Iterate through the link elements
        for link_element in link_elements:
            # Check if the title is present in the link element text
            if title in link_element.text:
                # Click on the link element
                link_element.click()
                break

        # Wait for the book page to load
        time.sleep(3)

        # Check if the author is present on the book page
        author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author}')]")
        if author_element:
            # Scrape the desired information
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
            gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
            avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
            count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
            count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
            date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

            # Scrape the genre
            genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
            genres = [genre.text.strip() for genre in genre_elements]
            genre1 = ', '.join(genres) if genres else None

            # Append scraped data to the self_published list
            self_published.append({
                'title': gr_title,
                'author': gr_author,
                'avg_rating': avg_rating,
                'count_rating': count_rating,
                'count_reviews': count_reviews,
                'date_published': date_published,
                'top_genre': genre1
            })
        else:
            # Handle the case when the author is not found on the book page
            unavailable.append({
                'title': title,
                'author': author
            })

        counter += 1  # Increment the counter after each iteration

    except Exception as e:
        # Handle the exception here
        unavailable.append({
            'title': title,
            'author': author
        })



In [47]:
len(self_published)

10

In [48]:
len(unavailable)

6

In [49]:
unavailable

[{'title': 'Craving', 'author': 'Helen Hardt'},
 {'title': 'Wyoming Heart', 'author': 'Diana Palmer'},
 {'title': 'State Of Terror',
  'author': 'Hillary Rodham Clinton and Louise Penny'},
 {'title': 'Kiss Of Christmas Magic',
  'author': 'Eve Langlais, Aubrey Rose, Molly Prince and others'},
 {'title': 'Wrong', 'author': 'Jana Aston'},
 {'title': 'A Feast For Crows', 'author': 'George RR Martin'}]

In [57]:
#still having issues with titles with parentheses.  try to use reg expression to remove content within parentheses from link element text before comparing to modified title


In [61]:
len(self_published)

10

In [59]:
len(unavailable)

5

In [60]:
unavailable

[{'title': 'The Arrangement 10', 'author': 'HM Ward'},
 {'title': 'Craving', 'author': 'Helen Hardt'},
 {'title': 'State Of Terror',
  'author': 'Hillary Rodham Clinton and Louise Penny'},
 {'title': 'Kiss Of Christmas Magic',
  'author': 'Eve Langlais, Aubrey Rose, Molly Prince and others'},
 {'title': 'Wrong', 'author': 'Jana Aston'}]

In [63]:
#still getting so many unavailable.  try adding more breaks?
import re

self_published = []
unavailable = []

# Convert unique_title_author_pairs to a list
unique_pairs_list = list(unique_title_author_pairs)

# Loop through the first 10 pairs
for title, author in unique_pairs_list[:10]:
    try:
        # Remove content within parentheses from the title
        title_without_parentheses = re.sub(r'\([^)]*\)', '', title).strip()

        # Look up the book page URL for the given title
        search_book = driver.find_element(By.NAME, "q")
        search_book.clear()  # Clear any previous search
        search_book.send_keys(title_without_parentheses)  # Use modified title as the search query
        search_book.submit()

        # Wait for search results to load
        time.sleep(3)

        # Find all the link elements on the search results page
        link_elements = driver.find_elements(By.TAG_NAME, "a")

        # Iterate through the link elements
        for link_element in link_elements:
            # Remove content within parentheses from the link element text
            link_text_without_parentheses = re.sub(r'\([^)]*\)', '', link_element.text).strip()

            # Check if the modified title is present in the modified link element text
            if title_without_parentheses.lower() in link_text_without_parentheses.lower():
                # Click on the link element
                link_element.click()
                break

        # Wait for the book page to load
        time.sleep(3)

        # Check if the author is present on the book page
        author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author}')]")
        if author_element:
            # Scrape the desired information
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
            gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
            avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
            count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
            count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
            date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

            # Scrape the genre
            genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
            genres = [genre.text.strip() for genre in genre_elements]
            genre1 = ', '.join(genres) if genres else None

            # Append scraped data to the self_published list
            self_published.append({
                'title': gr_title,
                'author': gr_author,
                'avg_rating': avg_rating,
                'count_rating': count_rating,
                'count_reviews': count_reviews,
                'date_published': date_published,
                'top_genre': genre1
            })
        else:
            # Handle the case when the author is not found on the book page
            unavailable.append({
                'title': title,
                'author': author
            })

    except:
        # Handle any exceptions that occur during the scraping process
        unavailable.append({
            'title': title,
            'author': author
        })


In [67]:
#maybe we should only check for authors last name on page, to avoid issues with punctuation and mutiple authors
# Check if the author's last name is present on the book page
self_published = []
unavailable = []

# Convert unique_title_author_pairs to a list
unique_pairs_list = list(unique_title_author_pairs)

# Loop through the first 10 pairs
for title, author in unique_pairs_list[:10]:
    try:
        # Remove content within parentheses from the title
        title_without_parentheses = re.sub(r'\([^)]*\)', '', title).strip()

        # Look up the book page URL for the given title
        search_book = driver.find_element(By.NAME, "q")
        search_book.clear()  # Clear any previous search
        search_book.send_keys(title_without_parentheses)  # Use modified title as the search query
        search_book.submit()

        # Wait for search results to load
        time.sleep(3)

        # Find all the link elements on the search results page
        link_elements = driver.find_elements(By.TAG_NAME, "a")

        # Iterate through the link elements
        for link_element in link_elements:
            # Remove content within parentheses from the link element text
            link_text_without_parentheses = re.sub(r'\([^)]*\)', '', link_element.text).strip()

            # Check if the modified title is present in the modified link element text
            if title_without_parentheses.lower() in link_text_without_parentheses.lower():
                # Click on the link element
                link_element.click()
                break

        # Wait for the book page to load
        time.sleep(3)

        # Check if the author is present on the book page
        author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author.split()[-1]}')]")
        if author_element:
            # Scrape the desired information
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
            gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
            avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
            count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
            count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
            date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

            # Scrape the genre
            genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
            genres = [genre.text.strip() for genre in genre_elements]
            genre1 = ', '.join(genres) if genres else None

            # Append scraped data to the self_published list
            self_published.append({
                'title': gr_title,
                'author': gr_author,
                'avg_rating': avg_rating,
                'count_rating': count_rating,
                'count_reviews': count_reviews,
                'date_published': date_published,
                'top_genre': genre1
            })
        else:
            # Handle the case when the author is not found on the book page
            unavailable.append({
                'title': title,
                'author': author
            })

    except:
        # Handle any exceptions that occur during the scraping process
        unavailable.append({
            'title': title,
            'author': author
        })


In [68]:
len(self_published)

9

In [69]:
len(unavailable)

1

In [70]:
unavailable

[{'title': 'Craving', 'author': 'Helen Hardt'}]